<a href="https://colab.research.google.com/github/Taka0007/2023-seminar-previous.term/blob/main/code/%E9%9F%B3%E5%A3%B0%E5%88%86%E9%9B%A2NMF%E5%AE%9F%E8%A3%85%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.decomposition import NMF

In [8]:
# wavファイルDL
!wget https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav -O aiueo.wav
file_path = '/content/aiueo.wav'
audio_data, sr = librosa.load(file_path)

--2023-06-20 08:00:20--  https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav [following]
--2023-06-20 08:00:20--  https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32832 (32K) [audio/wav]
Saving to: ‘aiueo.wav’

aiueo.wav           100%[===================>]  32.06K  --.-KB/s    in 0.002s  

2023-06-20 08:00:20 (15.0 MB/s) - ‘aiueo.wav’ saved [32832/32832]



<ipython-input-8-0da643ce9416>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [20]:
#高速フーリエ変換
fft_spec = np.fft.rfft(audio_data)

# NMFを行うために、fft_specを2次元配列に変換
fft_spec = fft_spec.reshape(len(fft_spec),1)
print(fft_spec)

[[-2.78108568e+01+0.00000000e+00j]
 [-4.17856923e+00+1.67983792e+01j]
 [ 2.63063313e+01+1.49804326e+01j]
 ...
 [ 6.29508110e-03-1.11692273e-06j]
 [ 6.29458866e-03+2.31351483e-06j]
 [ 6.29570565e-03-2.41347811e-07j]]


In [21]:
type(fft_spec)

numpy.ndarray

In [22]:
print(fft_spec)
print(fft_spec.shape)

[[-2.78108568e+01+0.00000000e+00j]
 [-4.17856923e+00+1.67983792e+01j]
 [ 2.63063313e+01+1.49804326e+01j]
 ...
 [ 6.29508110e-03-1.11692273e-06j]
 [ 6.29458866e-03+2.31351483e-06j]
 [ 6.29570565e-03-2.41347811e-07j]]
(54905, 1)


In [24]:
# 複素NMFの実装
def complex_nmf(V, rank, num_iterations):
    # 行列のサイズを取得
    n, m = V.shape

    # WとHの初期化
    W = np.random.rand(n, rank)
    H = np.random.rand(rank, m)

    # 反復更新
    for i in range(num_iterations):
        # Wの更新
        numerator_W = np.abs(V) @ H.T
        denominator_W = W @ H @ H.T
        W *= numerator_W / (denominator_W + 1e-9)

        # Hの更新
        numerator_H = W.T @ np.abs(V)
        denominator_H = W.T @ W @ H
        H *= numerator_H / (denominator_H + 1e-9)

    return W, H

rank = np.linalg.matrix_rank(fft_spec)
W, H = complex_nmf(fft_spec, rank, num_iterations=100)
Ans = W @ H
# 結果の表示
print("W:")
print(W)

print("H:")
print(H)

print('fft_spec')
print(fft_spec)

print('Ans')
print(Ans)


W:
[[1.29850093e+02]
 [8.08224713e+01]
 [1.41344550e+02]
 ...
 [2.93919845e-02]
 [2.93896868e-02]
 [2.93949001e-02]]
H:
[[0.21417664]]
fft_spec
[[-2.78108568e+01+0.00000000e+00j]
 [-4.17856923e+00+1.67983792e+01j]
 [ 2.63063313e+01+1.49804326e+01j]
 ...
 [ 6.29508110e-03-1.11692273e-06j]
 [ 6.29458866e-03+2.31351483e-06j]
 [ 6.29570565e-03-2.41347811e-07j]]
Ans
[[2.78108568e+01]
 [1.73102855e+01]
 [3.02727010e+01]
 ...
 [6.29507653e-03]
 [6.29458441e-03]
 [6.29570099e-03]]


## 参考資料

[【超初心者向け】複素NMFとは？更新式を丁寧に導出。](https://tips-memo.com/complex-nmf)